In [1]:
from tqdm import tqdm
import datetime as dt
import pymssql
import pandas as pd
import numpy as np
import warnings
import glob 
import os
import openpyxl
from io import BytesIO
from pandas import options
warnings.filterwarnings('ignore')

In [2]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Код склада], [Код товара], [Артикул поставщика], [Размер изделия], [Тип изделия 3], [Вставка Камней], [ID Сайта],[Название товара]
from [Staging].[Reports].[Remainings_8h]
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [3]:

stock.sample(5)

,Код склада,Код товара,Артикул поставщика,Размер изделия,Тип изделия 3,Вставка Камней,ID Сайта,Название товара
3580363,5872,ТОВ1062435,50-02-0001-22359,0.0,КРАСНЫЙ,ФИАНИТ БЕСЦВЕТ,4484502,"Подвес, фианит"
6233977,3075,ТОВ0726702,00030,0.0,ЧЁРНЫЙ,,4074186,Крест черн.
3708286,5128,ТОВ1258877,18-91-0000-26221#375,0.0,КРАСНЫЙ,,1130865,Серьги
3657507,3789,ТОВ1334477,129012140da,0.0,ЖЕЛТЫЙ,,1793864,"Серьги с зел. янтарем, поз."
3677101,4527,ТОВ1315523,901032076,0.0,БЕЛЫЙ С РОДИРОВАНИЕМ,,1491016,"Подвес мишка с роз., б/ц фиан. род."


In [4]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\ecom_stock_raw\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime) # мы ищем тут последний выгруженный файл (занимается выгрузкой Оризу Наталья email: Orizu.Natalya@zolotoy.ru)

df = pd.read_excel(latest_file)

In [5]:
latest_file # если последний файл - не за сегодняшнюю/вчерашнюю дату, стоит запросить актуальную выгрузку

'\\\\gold585.int\\uk\\Общее хранилище файлов\\Служба аналитики\\МЮР\\ecom_stock_raw\\Остатки 2024.07.24.xlsx'

In [6]:
df.sample(5)

,article,size,name,published,product_line,product_group,material,type3,weaving,fineness,avg_weight,price,discount_price,"Остатки, шт*",Unnamed: 14,Unnamed: 15,*без франчайзинговых магазинов и Румянцево
123631,2243551,17.0,Серебряное кольцо с фианитами и родированием,True,СИ,СИ КОЛЬЦО ЦВ/К,Серебро,ЧЁРНЫЙ С РОДИРОВАНИЕМ,NaN,925.0,2.43,2990,1790,9,NaN,NaN,NaN
74734,1727319,17.0,"Кольцо с зеленым кварцем гт и фианитами, с родир.",False,ПДК,ПДК КОЛЬЦА,Золото,КРАСНЫЙ,NaN,585.0,2.18,84270,29500,6,NaN,NaN,NaN
115419,2230094,NaN,Серебряная чайная ложка,True,СИ,СИ ПОСУДА,Серебро,ЧЁРНЫЙ,NaN,925.0,21.41,13020,7820,3,NaN,NaN,NaN
84308,1820811,20.0,Золотое кольцо с бриллиантами,True,ДК,ДК КОЛЬЦА,Золото,КРАСНЫЙ,NaN,585.0,1.80,100520,33170,2,NaN,NaN,NaN
91666,1893607,16.0,Золотое кольцо с бриллиантом и родированием с ...,True,ДК,ДК КОЛЬЦА,Золото,КРАСНЫЙ,NaN,585.0,1.44,61560,21620,3,NaN,NaN,NaN


In [7]:
stock['Размер изделия'] = stock['Размер изделия'].astype(str)
st = stock.groupby('ID Сайта',as_index=False).agg({'Код склада':lambda x: x.unique(),'Код товара':lambda x: x.unique(),'Тип изделия 3':lambda x: x.unique(),'Вставка Камней':lambda x: x.unique(),'Размер изделия':lambda x: x.unique()})
print(len(st))
nun = stock.groupby('ID Сайта',as_index=False).agg({'Тип изделия 3':lambda x: x.nunique(),'Вставка Камней':lambda x: x.nunique(),'Размер изделия':lambda x: x.nunique(),'Код склада':'count'})
print(len(nun))
st = st.merge(nun, left_on='ID Сайта', right_on = 'ID Сайта', how='inner')
print(len(st))

In [ ]:
st = st.drop(0).reset_index(drop=True)

In [ ]:
st.head(10)

,ID Сайта,Код склада_x,Код товара,Тип изделия 3_x,Вставка Камней_x,Размер изделия_x,Тип изделия 3_y,Вставка Камней_y,Размер изделия_y,Код склада_y
0,1000030,"[5890, 5892, 5653, 5765, 5961, 5864, 4760, 591...",[ТОВ1280314],[ПЛАТИНА],[],[0.0],1,1,1,44
1,1000054,"[7500, 5676, 5809]","[ТОВ1175938, ТОВ1175931, ТОВ1175933]",[КРАСНЫЙ],[ГРА+ФИАНИТ],"[20.5, 17.0, 18.0]",1,1,3,3
2,1000091,"[4363, 5890, 5897, 5899, 5976, 5087, 5781, 594...",[ТОВ01444969],[КРАСНЫЙ],[БРИЛЛИАНТ+ТАНЗАНИТ НАНО],[0.0],1,1,1,292
3,1000144,"[4363, 5890, 5986, 5891, 4656, 5897, 5899, 597...",[ТОВ0977759],[КРАСНЫЙ],[],[0.0],1,1,1,515
4,1000181,"[4899, 5749, 5698, 5627, 5641, 5674, 5840, 584...","[ТОВ1278199, ТОВ1278200, ТОВ1278196, ТОВ127819...",[ЧЁРНЫЙ],[],"[23.0, 24.0, 22.0, 23.5, 22.5, 18.0]",1,1,6,98
5,1000188,"[5890, 5887, 5898, 5897, 3426, 5892, 4871, 584...",[ТОВ1212796],[БЕЛЫЙ С РОДИРОВАНИЕМ],[],[0.0],1,1,1,791
6,1000204,[7211],[ТОВ0401165],[],[],[0.0],1,1,1,1
7,1000290,"[5899, 4910, 4466]",[ТОВ1126227],[ЦВЕТНОЙ],[],[0.0],1,1,1,5
8,1000302,[5442],[ТОВ1223939],[],[],[0.0],1,1,1,1
9,1000336,"[5277, 4688, 5772, 3284, 5633, 4826, 5916, 362...","[ТОВ1269894, ТОВ1269893, ТОВ1267348]",[БЕЛЫЙ С РОДИРОВАНИЕМ],[],"[20.0, 19.0, 21.0]",1,1,3,24


In [ ]:
st.columns = ['Артикул','Склады','ТОВ','Металл','Вставка','Размер изделия','Кол-во металлов','Кол-во вставок','Кол-во размеров','Общее кол-во']
st['Артикул'] = st['Артикул'].astype(int)

In [ ]:
# for i in tqdm(range(len(st))):
#     if st['Кол-во металлов'][i] == 1:
#         st.iloc[i]['Кол-во металлов'] = str(st['Кол-во металлов'][i])
#     if st['Кол-во вставок'][i] == 1:
#         st.iloc[i]['Кол-во вставок'] = str(st['Кол-во вставок'][i])
#     if st['Кол-во размеров'][i] == 1:
#         st.iloc[i]['Кол-во размеров'] = str(st['Кол-во размеров'][i])

In [ ]:
df.head(10)
df = df[~df['product_line'].str.contains('ФУТЛЯРЫ')]

,article,size,name,published,product_line,product_group,material,type3,weaving,fineness,avg_weight,price,discount_price,"Остатки, шт*",Unnamed: 14,Unnamed: 15,*без франчайзинговых магазинов и Румянцево
0,1000030,NaN,Серебряные серьги с фианитами и платинированием,True,СИ,СИ СЕРЬГИ РАЗН,Серебро,ПЛАТИНА,NaN,925.0,2.55,4550,2730,41,NaN,NaN,NaN
1,1000054,17.0,Золотое кольцо с гранатом и фианитами,True,ПДК,ПДК КОЛЬЦА,Золото,КРАСНЫЙ,NaN,375.0,1.25,20250,9520,1,NaN,NaN,NaN
2,1000054,18.0,Золотое кольцо с гранатом и фианитами,True,ПДК,ПДК КОЛЬЦА,Золото,КРАСНЫЙ,NaN,375.0,1.38,19780,9300,1,NaN,NaN,NaN
3,1000091,NaN,Золотая подвеска с танзанитом и бриллиантом,True,ДК,ДК ПОДВЕСКИ,Золото,КРАСНЫЙ,NaN,585.0,1.98,39660,18120,273,NaN,NaN,NaN
4,1000144,NaN,Футляр для ювелирных украшений Яблоко,False,ФУТЛЯРЫ,ФУТЛЯРЫ,NaN,КРАСНЫЙ,NaN,NaN,0.00,60,60,3288,NaN,NaN,NaN
5,1000181,18.0,Серебряное кольцо с чернением,True,СИ,СИ КОЛЬЦО БК,Серебро,ЧЁРНЫЙ,NaN,925.0,3.16,1990,1190,2,NaN,NaN,NaN
6,1000181,22.0,Серебряное кольцо с чернением,True,СИ,СИ КОЛЬЦО БК,Серебро,ЧЁРНЫЙ,NaN,925.0,3.58,1990,1190,3,NaN,NaN,NaN
7,1000181,22.5,Серебряное кольцо с чернением,True,СИ,СИ КОЛЬЦО БК,Серебро,ЧЁРНЫЙ,NaN,925.0,3.66,1990,1190,3,NaN,NaN,NaN
8,1000181,23.0,Серебряное кольцо с чернением,True,СИ,СИ КОЛЬЦО БК,Серебро,ЧЁРНЫЙ,NaN,925.0,3.13,1990,1190,7,NaN,NaN,NaN
9,1000181,24.0,Серебряное кольцо с чернением,True,СИ,СИ КОЛЬЦО БК,Серебро,ЧЁРНЫЙ,NaN,925.0,4.02,1990,1190,3,NaN,NaN,NaN


In [ ]:
df['published'] = df['published'].astype(str)
df['fineness'] = df['fineness'].astype(str)

In [ ]:
dff = df.groupby('article',as_index=False).agg({'name':'max','published':'max','product_line':'max','product_group':'max','type3':'max','fineness':'max'})

In [ ]:
dff.head(10)

,article,name,published,product_line,product_group,type3,fineness
0,1000030,Серебряные серьги с фианитами и платинированием,True,СИ,СИ СЕРЬГИ РАЗН,ПЛАТИНА,925.0
1,1000054,Золотое кольцо с гранатом и фианитами,True,ПДК,ПДК КОЛЬЦА,КРАСНЫЙ,375.0
2,1000091,Золотая подвеска с танзанитом и бриллиантом,True,ДК,ДК ПОДВЕСКИ,КРАСНЫЙ,585.0
3,1000144,Футляр для ювелирных украшений Яблоко,False,ФУТЛЯРЫ,ФУТЛЯРЫ,КРАСНЫЙ,nan
4,1000181,Серебряное кольцо с чернением,True,СИ,СИ КОЛЬЦО БК,ЧЁРНЫЙ,925.0
5,1000188,Серебряные детские серьги с фианитами и родиро...,True,СИ,СИ СЕРЬГИ РАЗН,БЕЛЫЙ С РОДИРОВАНИЕМ,925.0
6,1000290,Кулон новогодний Сказка близко из серебра с же...,False,ДФРС,ДФРС БИЖУТЕРИЯ,ЦВЕТНОЙ,nan
7,1000336,Кольцо с фиан. род.,False,СИ,СИ КОЛЬЦО ИФ,БЕЛЫЙ С РОДИРОВАНИЕМ,925.0
8,1000339,Золотые серьги с изумрудами и бриллиантами,True,ДК,ДК СЕРЬГИ,КРАСНЫЙ,585.0
9,1000409,"Комплект из керамики (кольцо+серьги) Серебро, ...",False,ДФРС,ДФРС БИЖУТЕРИЯ,БЕЛЫЙ,nan


In [ ]:
result = dff.merge(st, left_on='article', right_on = 'Артикул', how='inner')

In [ ]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Архив_отчеты\Продажи\По дням\\'
list_of_files = glob.glob(path + '*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

sklad = pd.read_excel(latest_file, sheet_name = 'Продажи по ассортименту', header = 2)
print(len(sklad))

1132


In [ ]:
latest_file

'\\\\gold585.int\\uk\\Общее хранилище файлов\\Служба аналитики\\Архив_отчеты\\Продажи\\По дням\\МОТИВАЦИОННЫЙ выполнение плана продаж июль 2024 - 24.xlsx'

In [ ]:
stock['Код склада'] = stock['Код склада'].astype('float64') 

sklad = sklad[['Unnamed: 0','Торговая марка', 'Юр лицо', 'Код склада', 'Дата открытия объекта', 'Код объекта', 'Город', 'Наименование объекта']]
sklad1 = sklad[['Код склада', 'Код объекта']].drop_duplicates().reset_index(drop=True)
sklad1 = sklad1[sklad1['Код объекта'].notna()]

sklad1['Код объекта'] = sklad1['Код объекта'].astype(str)
sklad1['8'] = sklad1['Код объекта'].apply(lambda x: x[0])

In [ ]:
sklad1 = sklad1[sklad1['8'] != '8']

stock1 = stock.merge(sklad1, left_on='Код склада', right_on = 'Код склада', how='inner')

stock1 = stock1.drop(columns=['8'])

In [ ]:
st1 = stock1.groupby('ID Сайта',as_index=False).agg({'Код товара':'count'})
st1.columns = ['Артикул','Кол-во без франчайзи']
st1 = st1.drop(0)
st1['Артикул'] = st1['Артикул'].astype(int)

In [ ]:
result = result.merge(st1,left_on='article', right_on = 'Артикул', how='inner')
result.drop(columns=['Артикул_x', 'Артикул', 'Артикул_y'], inplace =True)
result.rename(columns={'article': 'ID Сайта'}, inplace=True)

In [ ]:
result.sample(10)

,article,name,published,product_line,product_group,type3,fineness,Артикул_x,Склады,ТОВ,...,Вставка,Размер изделия,Кол-во металлов,Кол-во вставок,Кол-во размеров,Общее кол-во,Артикул_y,Кол-во без франчайзи_x,Артикул,Кол-во без франчайзи_y
59542,2948807,Кольцо из золота с фианитами,True,ИФ,ИФ КОЛЬЦА,КРАСНЫЙ,585.0,2948807,"[5277, 3268, 4528, 5772, 3209, 5442, 5124, 600...","[ТОВ1002363, ТОВ1002362, ТОВ1002366, ТОВ100236...",...,[ФИАНИТ БЕСЦВЕТ],"[18.5, 18.0, 20.0, 19.0, 17.5, 17.0, 16.5, 19.5]",1,1,8,19,2948807,19,2948807,11
65440,4019058,Серебряная печатка с фианитами и родированием,True,СИ,СИ КОЛЬЦО ПЕЧАТ,БЕЛЫЙ С РОДИРОВАНИЕМ,925.0,4019058,"[4363, 3443, 6531, 3742, 4850, 4805, 3598, 449...","[ТОВ0875208, ТОВ0753136, ТОВ0869042, ТОВ0753134]",...,[],"[20.5, 21.0, 18.0, 19.0]",1,1,4,53,4019058,53,4019058,50
21943,1518473,Женские часы Lee Cooper LC07326.390,True,ЧАСЫ,ЧАСЫ,NaN,nan,1518473,"[5698, 6159, 4487, 5891, 5845, 4383, 5916, 266...",[ТОВ1306002],...,[],[0.0],1,1,1,22,1518473,22,1518473,22
2815,1066862,Ремешок для часов,False,ЧАСЫ,ЧАСЫ,КРАСНЫЙ,nan,1066862,"[5715, 4688, 5627, 4565, 5920, 5862, 4441, 447...",[ТОВ1174768],...,[],[0.0],1,1,1,37,1066862,37,1066862,35
24856,1589727,Золотые серьги с топазами Sky,True,ПДК,ПДК СЕРЬГИ,КРАСНЫЙ,585.0,1589727,"[3713, 6020]",[ТОВ1240928],...,[ТОПАЗ],[0.0],1,1,1,2,1589727,2,1589727,2
55359,2243256,Кольцо с черным жемчугом,False,ПДК,ПДК КОЛЬЦА,КРАСНЫЙ,585.0,2243256,[4514],[ТОВ0627845],...,[ЖЕМЧУГ],[18.0],1,1,1,1,2243256,1,2243256,1
54349,2240614,Золотое кольцо с аметистом и фианитами,True,ПДК,ПДК КОЛЬЦА,КРАСНЫЙ,585.0,2240614,[5115],[ТОВ0739966],...,[АМЕТ+ФИАНИТ],[17.0],1,1,1,1,2240614,1,2240614,1
47947,2217947,Кольцо с брилл.,False,ДК,ДК КОЛЬЦА,БЕЛЫЙ,585.0,2217947,[5877],[ТОВ1105643],...,[БРИЛЛИАНТ],[18.0],1,1,1,1,2217947,1,2217947,1
89309,8354732,Серебряные серьги с жемчугом и эмалью,True,СИ,СИ ЖЕМЧУГ,БЕЛЫЙ С РОДИРОВАНИЕМ,925.0,8354732,[292],[ТОВ1178152],...,[],[0.0],1,1,1,1,8354732,1,8354732,1
67570,4280118,Золотой браслет Розочка,True,ЦБ,ЦБ БРАСЛЕТЫ,КРАСНЫЙ,585.0,4280118,"[3996, 5851, 5967, 3998, 5125, 5762]","[ТОВ0948342, ТОВ0948341]",...,[],"[18.0, 17.0]",1,1,2,6,4280118,6,4280118,4


In [ ]:
result.to_excel('dslkfgjdsklfjsdf.xlsx', index = False)